# 作业1

预测PM2.5值

## 1 数据预处理

### 1.1 导入训练数据

处理源数据，去掉不要的前3列，将降雨特征中的NR改成0，并将数据转化为numpy类型

### 1.2 将数据转化为18维特征和label

本来的表格 4320*24 每18行一天共480天，一天24个数据  
处理成连续的 12*18*480   
12个月，18维输入，每个月20天*24小时=480个数据

### 1.3 滑动窗口获得471个样本，并整合样本格式(162,1)
根据前9小时的数据预测第10小时的数据,  
每个月的20天都是连续的,  
每个月可以获得的sample：20\*24-10+1=471,  
12个月一共有471\*12=5652个sample,  
每个sample中,  
data由前9小时的18维数据组成，共9\*18=162个数据,  
label是第10小时的PM2.5值,  
下面处理把month_data处理成dataset和labels  

### 1.4 归一化
归一化，采用均值方差  
$ \frac{(x-\mu )}{S} $ 其中 $\mu$是均值，$S$是标准差  
抽取80%为训练集，20%为验证集

In [1]:
'''1.1'''
import sys
import pandas as pd
import numpy as np
import math

#导入训练数据
data=pd.read_csv('E:/dl2021/DL-hw/hw1/train.csv',encoding='big5')

#删掉前面3列的无效数据
data=data.iloc[:,3:]
#print(data)

#数据清洗，把NR换成0
data.replace('NR',0,inplace = True)

#转换成numpy数据类型
raw_data=data.to_numpy()
#print(raw_data)

'''1.2'''
month_data={}
for month in range(12):
    sample=np.empty([18,480])
    for day in range(20):
        sample[:,day*24:(day+1)*24]=raw_data[(day+month*20)*18:(day+1+month*20)*18,:]
    month_data[month]=sample

#print(month_data[0])

'''1.3'''
x=np.empty([471*12,9*18],dtype=float)
y=np.empty([471*12,1],dtype=float)
for month in range(12):
    for hour in range(471):
        x[month*471+hour,:]=month_data[month][:,hour:hour+9].reshape(-1)
        y[month*471+hour]=month_data[month][9,hour+9]

# print(x[1])

'''1.4'''
#归一化
#(x-μ)/S
x_mean=np.mean(x,axis=0)
x_std=np.std(x,axis=0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if x_std[j]!=0:
            x[i][j]=(x[i][j]-x_mean[j])/x_std[j]

'''
#从训练集里面20%作为验证集
div=math.floor(len(x)*0.8)
x_train_set=x[:div,:]
y_train_set=y[:div,:]
x_valid_set=x[div:,:]
y_vaild_set=y[div:,:]
print(y_train_set)
print(y_vaild_set)

x_train_set.shape
'''
x_train_set=x
y_train_set=y

## 2 处理测试集
处理测试集，每18行是一天，共239天，根据一天的前9小时数据预测第10小时

In [2]:
#导入测试数据
data_test=pd.read_csv('E:/dl2021/DL-hw/hw1/test.csv',header=None,encoding='big5')

#删掉前面2列的无效数据
data_test=data_test.iloc[:,2:]
#print(data)

#数据清洗，把NR换成0
data_test.replace('NR',0,inplace = True)

#转换成numpy数据类型
raw_test=data_test.to_numpy()
#print(raw_test)
#print(raw_test.shape)

#把240个样本转换成18*9=162维
x_test_set=np.empty([240,162],dtype=float)
for i in range(240):
    x_test_set[i,:]=raw_test[i*18:i*18+18,:].reshape(-1)

#print(x_test_set)

#归一化
#(x-μ)/S
x_test_mean=np.mean(x_test_set,axis=0)
x_test_std=np.std(x_test_set,axis=0)
#print(x_test_mean.shape,x_test_std.shape,x_test_set.shape)


for i in range(240):
    for j in range(162):
        if x_test_std[j]!=0:
            x_test_set[i][j]=(x_test_set[i][j]-x_test_mean[j])/x_test_std[j]
            
#print(x_test_set)

#拼接test_set，维度为（240，163），即在补上bias结点，默认特征为1
x_test_set=np.concatenate(
    (np.ones([len(x_test_set),1]),x_test_set),axis=1).astype(float)

#print(x_test_set)

___

# 2 training开始训练
确定超参数：步长（学习速率）、迭代次数  
## 计算损失函数 L
采用均方根误差，是预测值$wx$与真实值$y$偏差的平方与观测次数$num$比值的平方根  
$L=\sqrt{\frac{\sum (xw-y)^{2}}{num}}$
## 计算梯度 gradient
$\frac{\partial L}{\partial w}=\frac{x(xw-y)}{L\times num}$
## 梯度下降法迭代权重
$\epsilon$是为了防止adagrad的分母为0加上的极小数值  
$w^{t+1}=w^{t}-\frac{\eta }{\sqrt{\sum_{i=0}^{t}\left ( g^{i} \right )^{2}+\epsilon}}g^{t}$


In [3]:
#确定超参数
#维度dimention，18维*9天的数据+1个bias=163
dim=18*9+1
#初始化w，维度为 (163,1)
w=np.zeros([dim,1])
#拼接training_set，维度为（4521，163），即在补上bias结点，默认特征为1
x_train_set=np.concatenate(
    (np.ones([len(x_train_set),1]),x_train_set),axis=1).astype(float)
#设置学习率
learning_rate=1
#设置迭代次数
iter_time=10000
#初始化adagrads为0
adagrad=np.zeros([dim,1])
eps=0.0000000001

#x_train_set.shape

1 计算损失函数 L


$\  L=\sqrt{\frac{\sum (xw-y)^{2}}{num}}$


2 计算梯度 gradient


$\  \frac{\partial L}{\partial w}=\frac{x(xw-y)}{L\times num}$


3 梯度下降法迭代权重


$\  w^{t+1}=w^{t}-\frac{\eta }{\sqrt{\sum_{i=0}^{t}\left ( g^{i} \right )^{2}+\epsilon}}g^{t}$

In [4]:
#训练
for t in range(iter_time):
    #gradient descent
    #计算损失函数
    loss=np.sqrt(
        np.sum(np.power(np.dot(x_train_set,w)-y_train_set,2))/len(x_train_set))
    if(t%100==0):
        #每一百次迭代输出一次结果
        print("迭代次数{}:Loss函数值{}".format(t,loss))
    #计算梯度
    #gradient=2*np.dot(x_train_set.transpose(),np.dot(x_train_set,w)-y_train_set)
    gradient=np.dot(x_train_set.transpose(),
                    np.dot(x_train_set,w)-y_train_set)/(loss*len(x_train_set))
    adagrad+=gradient**2
    #梯度下降
    w=w-learning_rate*gradient/np.sqrt(adagrad+eps)
#w

迭代次数0:Loss函数值27.071214829194115
迭代次数100:Loss函数值8.52890603669083
迭代次数200:Loss函数值6.3363306210388135
迭代次数300:Loss函数值6.083741337773214
迭代次数400:Loss函数值5.936486495982032
迭代次数500:Loss函数值5.87608766207828
迭代次数600:Loss函数值5.832315016461716
迭代次数700:Loss函数值5.798104044728312
迭代次数800:Loss函数值5.773477125035168
迭代次数900:Loss函数值5.7553196488176885
迭代次数1000:Loss函数值5.741351607432848
迭代次数1100:Loss函数值5.730252315608886
迭代次数1200:Loss函数值5.72130474870991
迭代次数1300:Loss函数值5.714091726014985
迭代次数1400:Loss函数值5.708297261204126
迭代次数1500:Loss函数值5.703646231032833
迭代次数1600:Loss函数值5.699904775414663
迭代次数1700:Loss函数值5.6968836308395225
迭代次数1800:Loss函数值5.694434115783584
迭代次数1900:Loss函数值5.692441238309805
迭代次数2000:Loss函数值5.690817938199979
迭代次数2100:Loss函数值5.689503185024127
迭代次数2200:Loss函数值5.688466922672634
迭代次数2300:Loss函数值5.687716642823892
迭代次数2400:Loss函数值5.6872348196421125
迭代次数2500:Loss函数值5.686757210614006
迭代次数2600:Loss函数值5.686012470350578
迭代次数2700:Loss函数值5.685260895254458
迭代次数2800:Loss函数值5.684711627859442
迭代次数2900:Loss函数值5.684325

array([[ 2.13740269e+01],
       [ 3.82600885e-01],
       [-8.62079840e-01],
       [ 1.12874236e+00],
       [-1.65259941e+00],
       [-5.04671577e-02],
       [ 1.01441145e+00],
       [-3.84586589e-01],
       [-1.68559711e+00],
       [ 2.01865424e+00],
       [-3.19956979e-01],
       [ 1.44477793e-01],
       [ 8.17072668e-02],
       [ 9.37738762e-02],
       [-1.68307601e-02],
       [-2.46665714e-02],
       [-1.81623119e-01],
       [ 1.62078523e-01],
       [ 5.15781311e-01],
       [ 5.26202837e-02],
       [-2.61835640e-02],
       [ 6.54125051e-02],
       [-1.60409302e-01],
       [ 1.52244277e-01],
       [-2.07917587e-02],
       [-1.64662677e-01],
       [ 7.83877577e-02],
       [ 4.03905845e-01],
       [-3.12231460e-01],
       [ 3.81915655e-01],
       [-2.58608425e-01],
       [ 2.91951588e-01],
       [ 3.22531985e-01],
       [-5.17913993e-01],
       [ 8.84950629e-02],
       [ 1.78543882e-01],
       [ 8.09389060e-02],
       [ 9.04731359e-02],
       [ 6.6

In [5]:
np.save('weight.npy', w)

## 3 预测测试集

训练完成，接下来在验证集上验证模型
将预测结果存储到CSV文件

In [6]:
#预测
predict_y=np.dot(x_test_set,w)
#print(predict_y)

#将预测结果存储到CSV文件
import csv
with open('submit.csv',mode='w',newline='') as submit_file:
    csv_writer=csv.writer(submit_file)
    header=['id','value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row=['id_'+str(i),predict_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 4.292833067456147]
['id_1', 14.373494656768624]
['id_2', 17.721634429236296]
['id_3', 5.357966374417984]
['id_4', 20.82147838420883]
['id_5', 16.555737971561037]
['id_6', 17.822219583942587]
['id_7', 23.49073371397133]
['id_8', 13.03798641976596]
['id_9', 46.92257300225636]
['id_10', 8.709727680165948]
['id_11', 6.596968414537493]
['id_12', 49.18099903077797]
['id_13', 41.186255969411214]
['id_14', 16.38641995299686]
['id_15', 8.895647160617575]
['id_16', 24.991910988233876]
['id_17', 51.74389861704947]
['id_18', -2.247078051225884]
['id_19', 12.527906968919755]
['id_20', 32.69374037410227]
['id_21', 56.18145315669767]
['id_22', 7.0343268406175135]
['id_23', 13.825878348301844]
['id_24', 10.967055483192446]
['id_25', 29.11090725490343]
['id_26', 11.17707986109438]
['id_27', 50.88766667569088]
['id_28', 4.814494651288271]
['id_29', 43.50468356594277]
['id_30', 18.68369496349019]
['id_31', 5.451466341979973]
['id_32', 2.3915266075820227]
['id_33', 13.753820398808

___
## 4 模型评估

```
w=np.load('weight.npy')
#拼接training_set，维度为（1131，163），即在补上bias结点，默认特征为1
x_valid_set=np.concatenate(
    (np.ones([len(x_valid_set),1]),x_valid_set),axis=1).astype(float)
x_valid_set.shape

#计算损失函数
y_valid_predict=np.dot(x_valid_set,w)
loss_valid=np.sqrt(
    np.sum(np.power(y_valid_predict-y_vaild_set,2))/len(x_valid_set))
print("the Loss on valid set is {}".format(loss_valid))
for i in range(20):
    print("预测值为：{}，真实值为：{}".format(y_valid_predict[i],y_vaild_set[i]))
```